## Load CSV from spotify

In [1]:
import pandas as pd
import numpy as np

In [15]:
df_spotify = pd.read_csv('SpotifyFeatures.csv')

In [16]:
Genre_to_remove = ['Anime', "Children's Music", 'Children’s Music', 'Classical', 'Soundtrack', 'Opera', 'A Capella', 'Movie']
for genre in Genre_to_remove:
    df_spotify = df_spotify[df_spotify.genre != genre]

In [17]:
to_search = df_spotify[['artist_name', 'track_name', 'popularity', 'genre']]

We need to remove all the occurences of feat

In [18]:
def remove_feat(track_name):
    if '(' in track_name:
        track = track_name.split('(')
        return track[0]
    else:
        return track_name
    
def remove_space(track_name):
    if ' - ' in track_name:
        track = track_name.split(' - ')
        return track[0]
    else:
        return track_name

In [19]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_feat(x))

C:\Users\Alexis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
to_search = to_search.drop_duplicates()

In [21]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_space(x))

In [22]:
# save df to pickle
to_search.to_pickle("to_search.pkl")

In [2]:
# read df from pickle
to_search = pd.read_pickle("to_search.pkl")
to_search = to_search.reset_index().drop(['index'],axis=1)

In [12]:
to_search[to_search.track_name == 'No Sympathy']

,artist_name,track_name,popularity,genre
89533,Peter Tosh,No Sympathy,38,Reggae
91737,The Wailers,No Sympathy,31,Reggae
91882,Bob Marley & The Wailers,No Sympathy,29,Reggae
139051,The Wailers,No Sympathy,31,Ska


In [13]:
to_search = to_search[to_search.index >= 89533]

In [14]:
to_search

,artist_name,track_name,popularity,genre
89533,Peter Tosh,No Sympathy,38,Reggae
89534,Summer Thieves,Love Is Lost,47,Reggae
89535,Matisyahu,Close My Eyes,36,Reggae
89536,Slightly Stoopid,Drink Professionally,31,Reggae
89537,Wookiefoot,Loose Your Mind,33,Reggae
...,...,...,...,...
173883,Slave,Son Of Slide,39,Soul
173884,Jr Thomas & The Volcanos,Burning Fire,38,Soul
173885,Muddy Waters,,47,Soul
173886,R.LUM.R,With My Words,44,Soul


## Load musics from Genius

In [6]:
import requests
from bs4 import BeautifulSoup


def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + '6F7cdWVweWfmPXo-Nm-weqZ8_ouzvIk_ZfEXpNojIBdimY6ZI_e2zjPQiCcIU6YT'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html5lib')

    return html

Test by directly create different rows for different writers of the same song

In [8]:
# if already created
artists_writers_df = pd.read_pickle("artists_writers_df.pkl")
artists_writers_df = artists_writers_df.drop_duplicates(['track_name', 'artist'])

In [46]:
artists_writers_df = pd.DataFrame(columns=['track_name', 'artist', 'genre', 'popularity', 'more_infos'])

In [10]:
counter = 0
for row in to_search.iterrows():
    artist = row[1]['artist_name']
    song = row[1]['track_name']
    popularity = row[1]['popularity']
    genre = row[1]['genre']
    
    r = request_song_info(song, artist)
    content = r.json()
    if len(content['response']['hits']) != 0:
        counter +=1
        true_content = content['response']['hits'][0]['result']

        url = true_content['url']
        soup2 = scrap_song_url(url)

        infos = soup2.find_all('meta')[-1]

        print(song + ', ' + artist + ', ' + str(popularity))
        to_add = {'track_name': song, 'artist': artist,\
                  'popularity': popularity, 'genre': genre, \
                  'more_infos':(str(infos).replace('&quot;', '').split('label:')[1:])}
        
        artists_writers_df = artists_writers_df.append(to_add, ignore_index=True)
        
        if counter%1000 ==0:
            # save df to pickle
            print('saving ...')
            artists_writers_df.to_pickle("artists_writers_df.pkl")
            print('saved')

Till I Die, Machine Gun Kelly, 68
How It Is , Roddy Ricch, 72
Yours Truly, Austin Post, Post Malone, 69
Talk Up , Drake, 70
Yamborghini High, A$AP Mob, 71
Far Gone , Ski Mask The Slump God, 69
Numb, 21 Savage, 70
Stick to the Models, Future, 69
Best Day Of My Life, American Authors, 77
Lemme Freak, Lil Dicky, 66
Hurt You, The Weeknd, 73
Know Yourself, Drake, 69
Numb / Encore, JAY Z, 75
Time For That, Kevin Gates, 66
Survival, Drake, 70
Straight Outta Compton, N.W.A., 71
Too Deep, Kehlani, 71
Morning, Marc E. Bassy, 69
U're Mine, Kina, 71
A$AP Forever, A$AP Rocky, 73
Say Something, Justin Timberlake, 66
Dead And Gone , T.I., 70
Bae, O.T. Genasis, 68
Hit This Hard, Post Malone, 68
Get Right Witcha, Migos, 70
Know Me, NAV, 71
Lovin' , PnB Rock, 67
YSIV, Logic, 68
Good Life , G-Eazy, 65
Sandra’s Rose, Drake, 70
Poison, Brent Faiyaz, 69
U Said, Lil Peep, 72
Through The Wire, Kanye West, 68
My Love , Justin Timberlake, 72
NEVER, JID, 70
Gives You Hell, The All-American Rejects, 71
Let Me Blo

Modern Slavery, Huncho Jack, 68
Kids See Ghosts, KIDS SEE GHOSTS, 68
Dreams, NF, 67
Down With the Sickness, Disturbed, 73
Cold, Post Malone, 68
My Dawg, Lil Baby, 66
Banana Clip, Miguel, 67
Timeless , A Boogie Wit da Hoodie, 65
Nikki, Logic, 65
Issues , PnB Rock, 65
Everybody Dies, Logic, 66
Have Mercy, YBN Cordae, 64
Great Man, Kevin Gates, 64
Fuck Tha Police, N.W.A., 69
SMASH!, XXXTENTACION, 69
U and I, Ski Mask The Slump God, 67
Womanizer, Britney Spears, 75
Up All Night, Drake, 64
Toot It And Boot It, YG, 63
Talk Shit Like a Preacher, Future, 68
Knockout, Yung Gravy, 67
B.Y.O.B., System Of A Down, 75
Down with the Sickness, Disturbed, 72
Battle Cry, Polo G, 64
Marmalade , Macklemore, 70
Ridin , Yung Bans, 71
Transgression, Kodak Black, 66
So Good , Big Sean, 68
Lost In The Moment, NF, 67
Caribou Lou, Tech N9ne, 63
Salt Shaker, Ying Yang Twins, 66
The Monster, Eminem, 76
Passin' Me By, The Pharcyde, 68
Good Thing Go, Quinn XCII, 64
Lonely, Speaker Knockerz, 67
The Games We Play, Pus

I Was Never There, The Weeknd, 71
Shake That, Eminem, 74
Fasho Fasho , Lil Pump, 70
ZIPPER, BROCKHAMPTON, 66
Next Song, DaBaby, 68
Lick , Cardi B, 65
PettyWap, Young M.A, 66
Keep Their Heads Ringin', Dr. Dre, 62
Do It Again, Pia Mia, 76
Don't Wanna Breathe, Kodak Black, 65
In The Night, The Weeknd, 71
123, Smokepurpp, 68
Big Girls Don't Cry , Fergie, 75
Blem, Drake, 69
Angel, Shaggy, 65
Sittin' Sidewayz , Paul Wall, 64
Beware, Big Sean, 64
On The Run , Young Thug, 66
Something New , Wiz Khalifa, 65
Gangsta Nation, Westside Connection, 69
Honey, I'm Good., Andy Grammer, 70
Everything I Am, Kanye West, 61
Baptized In Fire, Kid Cudi, 63
Mad Stalkers, 21 Savage, 66
Glamorama, Bryce Vine, 62
Blunt Blowin, Lil Wayne, 61
It's Been Awhile, Staind, 65
The Devil In I, Slipknot, 72
Down, Jay Sean, 70
Friend Zone, A Boogie Wit da Hoodie, 64
DJ Got Us Fallin' In Love , Usher, 76
Fire, KIDS SEE GHOSTS, 66
Pull The Trigger, Russ, 66
666 , YG, 63
Superstar , Lupe Fiasco, 67
LLC, Nicki Minaj, 65
Peache

Come Thru, Drake, 62
D.T.B., A Boogie Wit da Hoodie, 63
Wasteland, 10 Years, 65
Party Rock Anthem, LMFAO, 65
No Air , Jordin Sparks, 74
Out the Mud, Kevin Gates, 62
Lonely, Yung Bans, 69
BDE Bonus, Mac Miller, 61
TOGETHER, Aminé, 65
Thoughts from a Balcony, Mac Miller, 62
schizophrenia, XXXTENTACION, 67
Cut The Cord, Shinedown, 67
SANTA MONICA, Aries, 64
Sunflower Seeds, Bryce Vine, 61
SUMO | ZUMO, Denzel Curry, 67
Selfish, Future, 68
Iron & Steel, Quinn XCII, 63
Happy, Pharrell Williams, 73
Bitchuary, Shordie Shordie, 63
Babies , KYLE, 64
Hillbilly, Upchurch, 62
Move Along, The All-American Rejects, 67
WRLD On Drugs, Future, 66
Congratulations, Drake, 66
Chum, Earl Sweatshirt, 66
Location, Playboi Carti, 65
The Old Us, Hopsin, 62
Because I Got High, Afroman, 67
March 14, Drake, 66
Fix This, Russ, 65
No Sleep Leak, Lil Uzi Vert, 61
Party Up, DMX, 70
Style Stealer , Gunna, 64
STAR, BROCKHAMPTON, 64
Hail Mary, 2Pac, 62
Ruff Ryders' Anthem, DMX, 67
Down For Me, G-Eazy, 64
CAROUSEL, Aries,

Run It, Logic, 61
The Life, Wiz Khalifa, 63
Fuck You , Yo Gotti, 62
i, Kendrick Lamar, 66
American Dream , Jeezy, 63
Deadroses, blackbear, 64
Spar , Dreezy, 65
Yah Mean, Playboi Carti, 64
I Don't Love You Anymore, Sarcastic Sounds, 67
Please Never Fall in Love Again, Ollie MN, 64
Honestly , G Herbo, 63
So Appalled, Kanye West, 62
Drinking About You, Ryan Oakes, 67
My Drip, Lil Baby, 62
Blame It, Jamie Foxx, 69
Fake Denim, Quinn XCII, 62
Wish the World Away, Ollie MN, 65
D'Evils, SiR, 67
One Way , 6LACK, 66
Today, DaBaby, 64
Whatever You Need , Meek Mill, 64
Beautiful, Akon, 64
Bounce, Flatbush Zombies, 64
Knock You Down, Keri Hilson, 67
Free, 6LACK, 67
Look Alive, Rae Sremmurd, 66
Blasé , Ty Dolla $ign, 62
Not The Only One, Kevin Gates, 61
Opps , Vince Staples, 67
Meet Again, Maxo Kream, 64
Cheatham County, Upchurch, 61
On My Level , Wiz Khalifa, 65
Lean 4 Real , Playboi Carti, 65
Used to, Josh A, 64
Ima Boss, Meek Mill, 65
Hey Mama, Kanye West, 61
D Rose, Lil Pump, 69
Heavy , Linkin P

...

Lovers Leap, Bim Sherman, 42
Summertime Is In Our Hands, Michael Franti & Spearhead, 37
Nothing At All, Iration, 36
Wild World, Maxi Priest, 36
Longshot, Passafire, 36
Antonia, Gondwana, 39
All This Time, Iration, 36
Waiting In Vain, Bob Marley & The Wailers, 42
Office, Pepper, 35
R.O.M.P., Tomorrows Bad Seeds, 37
Herb Stock , Tribal Seeds, 36
Redemption Song, Bob Marley & The Wailers, 45
Is It Worth It? , Damian Marley, 43
Hurt Me Tomorrow, K'NAAN, 40
Land of Love, The Green, 37
Karma Police, Easy Star All-Stars, 48
Young Lover, Cocoa Tea, 46
Libertad, Tribal Seeds, 36
Prisoner, 311, 42
Oh What A Feeling, Wailing Souls, 40
I Don't Wanna Go, Ballyhoo!, 38
Crisis, Bob Marley & The Wailers, 44
Bump, Rebelution, 37
Electric Avenue, Skindred, 44
Straighter Roads, Julian Marley, 45
Hip Hop Misfits, Dirty Heads, 34
Easy, Dirty Heads, 34
Rebel Music, Bob Marley & The Wailers, 41
Summer Breeze, SOJA, 43
It's Not Easy, Lucky Dube, 45
Jah Atmosphere, SOJA, 48
Show Me the Road, Katastro, 37
Redem

Laberintos, Dread Mar I, 57
Take Me Alive, Michael Franti & Spearhead, 35
Dancehall Days, The Beautiful Girls, 34
Mercy, 10 Ft. Ganja Plant, 36
Righteous Ways, Kings and Comrades, 36
Backas, Mr. Vegas, 9
Boombastic, Shaggy, 42
Never Sober, Popcaan, 40
Always & Forever, The Green, 35
Melody Of A Broken Heart, HIRIE, 35
Beautiful Day, Ballyhoo!, 35
Big Mistake, Pepper, 35
Sand Dollars, 311, 41
Storm Boy, Xavier Rudd, 51
Blue Dreamz, Collie Buddz, 38
Keep On Moving, Bob Marley & The Wailers, 37
One Bright Day , Slightly Stoopid, 35
Look Who's Dancin', Ziggy Marley & The Melody Makers, 48
Red Red Wine, UB40, 36
Sube el Humo, Cultura Profética, 51
We Party, The Holdup, 33
My Missy, Part One Tribe, 35
Rasta, Refuse It, Tribal Seeds, 35
Girls., Ballyhoo!, 35
You Girl , Shaggy, 47
Strong Me Strong, Yellowman, 42
Headphones, Dirty Heads, 32
Row Fisherman, Wailing Souls, 36
I Chase The Devil, Max Romeo, 46
My Only , Unified Highway, 34
saving ...
saved
Iya Lovin', Iya Terra, 36
Running With A Gu

Marijuanaman, Ziggy Marley, 33
Windylike, Dispatch, 38
Open My Eyes, KBong, 35
Good Life, Cocoa Tea, 39
I'm Not Crazy, Don Carlos, 40
Lost Paradise, Tribal Seeds, 34
Why Am I A Rastaman?, Culture, 47
Kill the Power, Skindred, 41
Dance Of The Vampires, Roots Radics, 38
Mad Me, Popcaan, 41
More Than Ever, Rebelution, 34
Scars On My Feet, Stephen Marley, 36
Mr. Nine, Buju Banton, 36
Turtle Racing , The Expanders, 33
, Aer, 36
Mystical Reggae , Alborosie, 46
Right Away, The Holdup, 32
Begin Again, Dispatch, 38
Here We Go, Dispatch, 38
Grandes pasos, Movimiento Original, 61
Peace in Your Mind , Bob Andy, 45
The End, Iration, 33
Reflect, Tomorrows Bad Seeds, 34
You Don't Know, Iration, 33
Guns in Paradise, Slightly Stoopid, 34
Ocean, Slightly Stoopid, 32
Waitin', Los Pericos, 55
RePResentando, Pj Sin Suela, 35
Kinky Reggae, Bob Marley & The Wailers, 41
Appreciate You, MAGIC!, 44
Island Home, KBong, 29
Anything You Ask For, House Of Shem, 46
We A De Rasta, Israel Vibration, 40
Peaceizafiya, S

Notion , Tash Sultana, 34
Gift, The Movement, 32
Back Around, Iration, 33
Del tope al fondo, Cultura Profética, 50
Tenement Yard , Inner Circle, 40
Lost, Hours Eastly, 32
Coming up Empty, Matisyahu, 36
Cool Out Son, Junior Murvin, 38
Pomps & Pride, Toots & The Maytals, 38
Mad Mad World , Shaggy, 48
We Will Walk, Matisyahu, 32
Homebrew, 311, 38
Touch the Sky, Kings and Comrades, 34
Revolution, Bob Marley & The Wailers, 40
Blood and Fire, Niney The Observer, 41
My Girl, Sizzla, 39
Ain't No Sunshine, Ken Boothe, 45
My Satellite, Three Legged Fox, 33
Rum & Redbull, Beenie Man, 42
Quema Como el Fuego, Dread Mar I, 55
Love Mi Haffi Get, Beres Hammond, 36
Double Trouble, Beres Hammond, 36
Natural Mystic [feat. Ky Mani Marley], Alborosie, 47
The Heathen, Bob Marley & The Wailers, 41
Oxygen, Dirty Heads, 32
Stop That Train, Bob Marley & The Wailers, 40
Wear You To The Ball, UB40, 30
Don't Talk Serious, The Holdup, 31
Rescue, Mama Kin, 42
Daddy's $$$, Katastro, 33
Dos Y Siete, Vicente Garcia, 49

KeyboardInterrupt: 

In [15]:
# save df to pickle
artists_writers_df.to_pickle("artists_writers_df.pkl")

In [52]:
artists_writers_df = artists_writers_df.reset_index().drop(['index'], axis=1)

In [11]:
artists_writers_df

,track_name,artist,genre,popularity,more_infos
0,Be Without You,Mary J. Blige,R&B,65,"[Strings,artists:[{_type:artist,api_path:/arti..."
1,Desperado,Rihanna,R&B,63,"[Distributor,artists:[{_type:artist,api_path:/..."
2,Ice On My Baby,Yung Bleu,R&B,62,"[Video,overlay_compatible_image_url:https://im..."
3,Heaven Falls / Fall on Me,Surfaces,R&B,61,[]
4,Love Myself,Olivia O'Brien,R&B,68,"[Video,overlay_compatible_image_url:https://im..."
...,...,...,...,...,...
70006,Si el Amor Se Cae,Los Cafres,Reggae,54,[]
70007,Les copains d'abord,Georges Brassens,Reggae,36,[]
70008,Rice Water,Dispatch,Reggae,37,[]
70009,Visions,Dirty Heads,Reggae,33,[]
